In [11]:
from dataclasses import dataclass, field, asdict
from typing import List

import pandas as pd
import numpy as np

In [12]:
from  helpers_db import get_engine, run_sql
engine = get_engine()
with engine.begin() as conn:
  data = run_sql(conn, """SELECT * FROM transactions ORDER BY dt""")
  df = pd.DataFrame(data)
  df_txn = df[(df['category'] == 'buy') | (df['category'] == 'sell')]
  print(df_txn)


      id  broker_id   symbol                        dt  \
1    539  397981597     CSCO 2023-05-30 14:30:50+00:00   
2    538  399776840     PYPL 2023-06-02 15:55:12+00:00   
3    537  399776860     PYPL 2023-06-02 15:55:13+00:00   
4    536  403174115     CVNA 2023-06-12 18:39:14+00:00   
5    535  406987589     AMZN 2023-06-21 14:30:17+00:00   
..   ...        ...      ...                       ...   
213  437  531037227  EGLN.UK 2024-04-12 16:17:28+00:00   
214  498  531041052      LQQ 2024-04-12 16:19:44+00:00   
215  497  531043612     AAPL 2024-04-12 16:21:16+00:00   
216  496  531043644     AAPL 2024-04-12 16:21:18+00:00   
217  495  531045046       CI 2024-04-12 16:22:24+00:00   

                           comment     amount   quote  shares  shares_left  \
1               OPEN BUY 2 @ 50.24  100.48000   50.24  2.0000       2.0000   
2       OPEN BUY 0.55/1.55 @ 64.24   35.33200   64.24  0.5500       0.5500   
3          OPEN BUY 1/1.55 @ 64.23   64.23000   64.23  1.0000       1

In [16]:
@dataclass
class Transaction:
  dt: str = '2024-04-12 16:17:28+00'
  symbol: str = 'CSCO'
  shares: float = 1
  currency: str = 'usd'
  quote: float = 64.24
  amount: float = 64.24

In [17]:
@dataclass
class GeneralLedger:
  transactions: List[Transaction] = field(default_factory=list)

  def trial_balance(self):
    df = pd.DataFrame([asdict(txn) for txn in self.transactions])
    return df.pivot_table(index=['symbol','currency'],
                          values=['shares','quote','amount'],
                          aggfunc={'shares': np.sum, 'quote': 'last','amount': np.sum}).reset_index()

In [18]:
transactions: List[Transaction] = []
for index, row in df_txn.iterrows():
  if row['symbol'] != 'EUR.USD':
    transactions.append(Transaction(row['dt'],row['symbol'],row['shares'],row['currency'],row['quote'], row['amount']))
gl = GeneralLedger(transactions)
gl.trial_balance()

,symbol,currency,amount,quote,shares
0,AAOI,usd,1387.900000,17.3480,75.0000
1,AAPL,usd,3581.754168,176.5200,19.8318
2,ADP,usd,1516.380000,252.5800,0.0000
3,ALB,usd,1437.510000,114.5900,11.0000
4,AMZN,usd,609.734000,157.8790,4.0000
5,BABA,usd,819.725000,71.9680,11.0000
6,CI,usd,1284.300000,351.2000,4.0000
7,CLS,usd,2114.700000,44.7400,50.0000
8,CNDX.UK,usd,1083.345566,1029.7400,1.0616
9,CRDF,usd,1030.500000,5.4950,200.0000
